In [1]:
from gensim.models.ldamodel import LdaModel
from gensim.models.coherencemodel import CoherenceModel
from gensim.corpora import MmCorpus
from gensim.corpora import Dictionary
from collections import Counter, defaultdict
from numpy.random import seed
from metadata.metadata import ACL_metadata
from _topic_modeling.lda_loader import Loader
import _pickle as pkl
from metadata import Gender
import matplotlib.pyplot as plt
import numpy as np
import logging
import gensim 
seed(1)
logging.basicConfig(format='%(levelname)s : %(message)s', level=logging.INFO)
logging.root.level = logging.INFO

In [2]:
loader = Loader()

loaded corpus index from /home/ramona/Desktop/Honours-LDA/aan/../acl_bow.mm.index
initializing corpus reader from /home/ramona/Desktop/Honours-LDA/aan/../acl_bow.mm
accepted corpus with 23595 documents, 60434 features, 16454656 non-zero entries
loading LdaMallet object from /home/ramona/Desktop/Honours-LDA/aan/../malltepy
loaded /home/ramona/Desktop/Honours-LDA/aan/../malltepy


In [3]:
lda = gensim.models.ldamodel.LdaModel.load('../models/model500.pkl')

loading LdaModel object from ../models/model500.pkl
loading expElogbeta from ../models/model500.pkl.expElogbeta.npy with mmap=None
setting ignored attribute id2word to None
setting ignored attribute state to None
setting ignored attribute dispatcher to None
loaded ../models/model500.pkl
loading LdaModel object from ../models/model500.pkl.state
loaded ../models/model500.pkl.state


In [4]:
lda.id2word = loader.id2word

In [5]:
lda.get_document_topics(loader.corpus[1])

[(0, 0.025419349521797297),
 (6, 0.025850081552067954),
 (29, 0.034960361179327479),
 (39, 0.015666331484132281),
 (48, 0.017831854308786279),
 (54, 0.15960324726529973),
 (59, 0.056092641413856302),
 (61, 0.016340065358688621),
 (67, 0.10597677929556155),
 (70, 0.31895346351565473),
 (83, 0.1131325160571294),
 (90, 0.059135231314702316)]

In [6]:
import pyLDAvis.gensim as gensimvis
import pyLDAvis

In [7]:
len(loader.id2word.keys())

60434

In [8]:
corpus = loader.corpus
dictionary = loader.dic

In [9]:
vis_data = gensimvis.prepare(lda, corpus, dictionary)
pyLDAvis.display(vis_data)

/home/ramona/anaconda3/envs/mlp/lib/python3.6/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]


In [8]:
gensimcm = CoherenceModel(model=lda, corpus=corpus, dictionary=dictionary, coherence='u_mass')

In [13]:
gensimcm.get_coherence()

CorpusAccumulator accumulated stats from 1000 documents
CorpusAccumulator accumulated stats from 2000 documents
CorpusAccumulator accumulated stats from 3000 documents
CorpusAccumulator accumulated stats from 4000 documents
CorpusAccumulator accumulated stats from 5000 documents
CorpusAccumulator accumulated stats from 6000 documents
CorpusAccumulator accumulated stats from 7000 documents
CorpusAccumulator accumulated stats from 8000 documents
CorpusAccumulator accumulated stats from 9000 documents
CorpusAccumulator accumulated stats from 10000 documents
CorpusAccumulator accumulated stats from 11000 documents
CorpusAccumulator accumulated stats from 12000 documents
CorpusAccumulator accumulated stats from 13000 documents
CorpusAccumulator accumulated stats from 14000 documents
CorpusAccumulator accumulated stats from 15000 documents
CorpusAccumulator accumulated stats from 16000 documents
CorpusAccumulator accumulated stats from 17000 documents
CorpusAccumulator accumulated stats from

-1.6901845123922141

In [12]:
acl = ACL_metadata()
df = acl.meta_df

Remember to use acl.modeling_files and modeling_df for topic modeling


In [9]:
for i, p in lda.get_topic_terms(38):
    print(loader.id2word[i])

tile
tag
derivation
tree
grammar
ill
adjoin
elementary
elementary_tree
auxiliary


In [32]:
#doc_topics = list(map(lambda x: lda.get_document_topics(x), corpus))

In [10]:
# with open("../doc_topics_gensim.pkl","wb") as f:
#     pkl.dump(doc_topics, f)
with open("../doc_topics_gensim.pkl","rb") as f:
    doc_topics = pkl.load(f)

In [13]:
females_topics = []
females_topics_proportions = defaultdict(int) # P(topic|gender)'
males_topics_proportions = defaultdict(int)
fem_norm = 0 # normalizer
male_norm = 0
males_topics = []
for i, file in enumerate(acl.train_files):
    try:
        topics = doc_topics[i]
        gender = df.loc[acl.get_id(file)]["genders"][0] #first author
        if gender == Gender.female:
            for topic, p in topics:
                females_topics_proportions[topic] += p
            fem_norm +=1
        elif gender == Gender.male:
            for topic, p in topics:
                males_topics_proportions[topic] += p
            male_norm +=1
    except KeyError:
        pass

In [45]:
doc_topics[21]

[(0, 0.025664048800330137),
 (3, 0.1472034337219437),
 (8, 0.015802955455413117),
 (15, 0.0562709383421671),
 (25, 0.014178564005336527),
 (27, 0.011599366266778259),
 (39, 0.3283978219092855),
 (51, 0.018097345193582897),
 (64, 0.029926796385942953),
 (95, 0.298751496969276)]

In [14]:
for idx in range(100):
    f = (females_topics_proportions[idx] + 1e-10) / fem_norm
    m = (males_topics_proportions[idx] + 1e-10) / male_norm
    odds = (f * (1-f)) / (m * (1-m))
    if odds > 1.5:
        print("Females are {0:.2f} more likeky than men to write about topic {1:d}".format(odds, idx))
        print("Most common words for this topic are:")
        for w,p in lda.show_topic(idx,10):
            print(w,)
        print("-"*30)
        print("-"*30)


Females are 2.66 more likeky than men to write about topic 21
Most common words for this topic are:
dik
related
help
mark
extend
directly
additional
thank
final
focus
------------------------------
------------------------------
Females are 2.83 more likeky than men to write about topic 32
Most common words for this topic are:
czech_german
itu
rankboost
equiv
mph
bsl
gdp
altered
about 90% of
iwi
------------------------------
------------------------------


In [15]:
for idx in range(100):
    f = (females_topics_proportions[idx] + 1e-10) / fem_norm
    m = (males_topics_proportions[idx] + 1e-10) / male_norm
    odds = (m * (1-m)) / (f * (1-f))
    if odds > 1.5:
        print("Males are {0:.2f} more likeky than females to write about topic {1:d}".format(odds, idx))
        print("Most common words for this topic are:")
        for w,p in lda.show_topic(idx,10):
            print(w,)
        print("-"*30)
        print("-"*30)
